In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException
from pymongo import MongoClient

In [2]:
s = Service("./chromedriver.exe")
driver = webdriver.Chrome(service=s)
driver.implicitly_wait(5)

driver.get("https://www.mvideo.ru/")

while True:
    driver.find_element(By.TAG_NAME, "body").send_keys(Keys.PAGE_DOWN)
    
    try:
        button_trend = driver.find_element(By.XPATH, "//button[@class='tab-button ng-star-inserted']")
        button_trend.click()

        names_el = driver.find_elements(By.XPATH, "//mvid-shelf-group//div[@class='product-mini-card__name ng-star-inserted']")
        prices_el = driver.find_elements(By.XPATH, "//mvid-shelf-group//div[@class='product-mini-card__price ng-star-inserted']")
        links_el = driver.find_elements(By.XPATH, "//mvid-shelf-group//div[@class='product-mini-card__name ng-star-inserted']//a")
    except NoSuchElementException:
        continue
    else:
        break
             
names = []
prices = []
links = []

for name in names_el:
    names.append(name.text)

for price in prices_el:
    prices.append(price.text)

for link in links_el:
    links.append(link.get_attribute("href"))
    
for i in range(len(prices)):
    prices[i] = int("".join(prices[i].split()[:2]))

In [9]:
client = MongoClient("localhost", 27017)
db = client["mvideo"]
trends = db.trends

In [10]:
for i in range(len(names)):
    trends.insert_one({
        "Название": names[i],
        "Цена": prices[i],
        "Ссылка": links[i]
    })

In [11]:
list(trends.find({}))

[{'_id': ObjectId('6220a955f8d7e119f0746b08'),
  'Название': 'Смартфон ZTE Blade A51 2+32GB Gray',
  'Цена': 7999,
  'Ссылка': 'https://www.mvideo.ru/products/smartfon-zte-blade-a51-232gb-gray-30060503'},
 {'_id': ObjectId('6220a955f8d7e119f0746b09'),
  'Название': 'Ноутбук Acer Aspire 3 A315-34-C9WH NX.HE3ER.01V',
  'Цена': 34999,
  'Ссылка': 'https://www.mvideo.ru/products/noutbuk-acer-aspire-3-a315-34-c9wh-nxhe3er01v-30059596'},
 {'_id': ObjectId('6220a955f8d7e119f0746b0a'),
  'Название': 'Наушники True Wireless HIPER Lazo X17 розовый (HTW-LX17)',
  'Цена': 1699,
  'Ссылка': 'https://www.mvideo.ru/products/naushniki-true-wireless-hiper-lazo-x17-rozovyi-htw-lx17-50167156'},
 {'_id': ObjectId('6220a955f8d7e119f0746b0b'),
  'Название': 'Телевизор Novex NWX-55U169TSS',
  'Цена': 37999,
  'Ссылка': 'https://www.mvideo.ru/products/televizor-novex-nwx-55u169tss-10028919'},
 {'_id': ObjectId('6220a955f8d7e119f0746b0c'),
  'Название': 'Холодильник Haier CEF537ACG',
  'Цена': 46999,
  'Ссылка